# Features 

## Importation des librairies

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# ACP
import sklearn.decomposition as sd
import sklearn.preprocessing as sp
# FFT
from scipy.fftpack import fft
# Hierarchical clustering
import scipy.cluster.hierarchy as sch
#Wavelet
import pywt
from pywt import wavedec
from statsmodels.robust import mad
# LOF
import sklearn.neighbors as sn
# Isolation Forest
import sklearn.ensemble as se

# Plot et Display
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sb
sb.set()
sb.set_style("whitegrid")

## Importation des données

On lit les données d'apprentissage et de test et on forme une liste de toutes les données contenant au total 428 vols.

In [4]:
#path_appr = "../Donnees_projet/Ensemble_apprentissage/"
#path_test = "../Donnees_projet/Ensemble_test/"
#path_appr = "../../4A/Projet/Ensemble d'apprentissage/"
#path_test = "../../4A/Projet/Ensemble de test/"
path_test = "../DetecAnomal/Ensemble d'apprentissage/"
path_appr ="../DetecAnomal/Ensemble de test/"

In [5]:
# Liste des séries d'apprentissage
files = os.listdir(path_appr)
liste_appr = [pd.read_csv(path_appr + file) for file in files]
# Liste des séries de test
files = os.listdir(path_test)
liste_test = [pd.read_csv(path_test + file) for file in files]
# Liste complète
liste_comp = liste_appr + liste_test
n = len(liste_comp) 

On observe que toutes les séries se terminent par un fort pic. On fait le choix de l'enlever car il n'apporte rien de significatif et risque de perturber l'analyse. 

In [28]:
def enlever_pic(serie, seuil=10, pts_moy=2, plot=False) :
    """Retire le pic final, le seuil est le plus petit accroissement considéré comme appartenant
    encore au pic. Les accroissements sont moyennés sur plusieurs points pour ne pas être sensibles 
    aux possibles irrégularités dans le pic.
    L'option plot=True permet de vérifier graphiquement le résultat.
    Retourne le dataframe tronqué"""

    val_serie = serie["FF_1"].values
    if plot :
        plt.plot(val_serie)
        
    accr = abs(val_serie[1:] - val_serie[:-1])
    moy_accr = [np.mean(accr[i : i + pts_moy]) for i in range(len(accr) - pts_moy)]

    for ind, val in enumerate(moy_accr[::-1]) :
        if val < seuil and ind >= 1 :
            if plot :
                plt.plot(val_serie[:-ind], color="red")
                plt.show()
                
            return serie.drop(serie.index[-ind:])
            break
    else : 
        return serie

In [30]:
liste_comp = [enlever_pic(serie) for serie in liste_comp] 

## Liste des features que l'on veut  

- min, max
- mean 
- écart type 
- Skewness 
- Kurtosis 
- Energy 
- Average Crossing 
- Distribution spectrale d'énergie 
- Pourcentage d'extrema locaux 

https://arc.aiaa.org/doi/pdf/10.2514/6.2016-2430

### Min & Max 

In [31]:
listemin = [min(liste["FF_1"]) for liste in liste_comp]

In [46]:
#print(listemin)
#len(listemin)

On récupère les numéros de séries dont le min est à 0. 

In [35]:
listemin.index(0.0)

19

In [37]:
listemax = [max(liste["FF_1"]) for liste in liste_comp]

In [45]:
#print(listemax)
#len(listemax)

### Mean

In [42]:
listemean = [np.mean(liste["FF_1"]) for liste in liste_comp]

In [44]:
#print(listemean)

### Ecartype 

In [48]:
listestd = [np.std(liste["FF_1"]) for liste in liste_comp]

In [50]:
#print(listestd)